In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

In [2]:
# Load the data
data = pd.read_csv(r'.\SpotifyLyricsAnnotated.csv', sep='\t')

# Tokenize the lyrics using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 512

def tokenize_lyrics(lyrics):
    return tokenizer.encode_plus(
        lyrics,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

In [3]:
subset_data = data.iloc[:2000]

subset_data['text'] = subset_data['text'].apply(tokenize_lyrics)

c:\Users\manya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
subset_data['text'][0]

{'input_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[  101,  2298,  2227,  6919,  2227,  2965,  2242,  2569,  2298,
         2126,  8451,  5927,  5341,  2028,  3507,  2016,  2015,  2785,
         2611,  3084,  2514,  2986,  2071,  2412,  2903,  2071,  3067,
         2016,  2015,  2785,  2611,  2302, 10047,  2630,  2412,  3727,
         2071,  2071,  2175,  3328,  2380,  4324, 11025,  2015,  2192,
         2092,  2175,  3788,  2847,  3331,  2477,  2933,  2016,  2015,
         2785,  2611,  3084,  2514,  2986,  2071,  2412,  2903,  2071,
         3067,  2016,  2015,  2785,  2611,  2302, 10047,  2630,  2412,
         3727,  2071,  2071,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [8]:
distinct_labels = data['Emotion'].unique()

# Create a dictionary to map labels to numbers
emotion_mapping = {label: idx for idx, label in enumerate(distinct_labels)}
data['Emotion'] = data['Emotion'].map(emotion_mapping)

In [27]:
emotion_mapping

{'admiration': 0,
 'caring': 1,
 'gratitude': 2,
 'neutral': 3,
 'sadness': 4,
 'realization': 5,
 'love': 6,
 'amusement': 7,
 'fear': 8,
 'joy': 9,
 'anger': 10,
 'desire': 11,
 'approval': 12,
 'excitement': 13,
 'surprise': 14,
 'confusion': 15,
 'annoyance': 16,
 'remorse': 17,
 'optimism': 18,
 'embarrassment': 19,
 'curiosity': 20,
 'nervousness': 21,
 'disgust': 22,
 'pride': 23,
 'grief': 24,
 'disappointment': 25,
 'disapproval': 26,
 'relief': 27}

In [22]:
"""
# Split the data into train and test sets
train_data, test_data = train_test_split(subset_data, test_size=0.2, random_state=42)

# Tokenize the lyrics
train_lyrics = [tokenize_lyrics(lyrics) for lyrics in train_data['text']]
test_lyrics = [tokenize_lyrics(lyrics) for lyrics in test_data['text']]

# Convert emotion labels to numpy arrays
train_labels = np.array(train_data['Emotion'])
test_labels = np.array(test_data['Emotion'])

# Convert the datasets to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': np.concatenate([np.array([lyrics['input_ids']]) for lyrics in train_lyrics]),
    'attention_mask': np.concatenate([np.array([lyrics['attention_mask']]) for lyrics in train_lyrics])
}, train_labels))

test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': np.concatenate([np.array([lyrics['input_ids']]) for lyrics in test_lyrics]),
    'attention_mask': np.concatenate([np.array([lyrics['attention_mask']]) for lyrics in test_lyrics])
}, test_labels))
"""

"\n# Split the data into train and test sets\ntrain_data, test_data = train_test_split(subset_data, test_size=0.2, random_state=42)\n\n# Tokenize the lyrics\ntrain_lyrics = [tokenize_lyrics(lyrics) for lyrics in train_data['text']]\ntest_lyrics = [tokenize_lyrics(lyrics) for lyrics in test_data['text']]\n\n# Convert emotion labels to numpy arrays\ntrain_labels = np.array(train_data['Emotion'])\ntest_labels = np.array(test_data['Emotion'])\n\n# Convert the datasets to TensorFlow datasets\ntrain_dataset = tf.data.Dataset.from_tensor_slices(({\n    'input_ids': np.concatenate([np.array([lyrics['input_ids']]) for lyrics in train_lyrics]),\n    'attention_mask': np.concatenate([np.array([lyrics['attention_mask']]) for lyrics in train_lyrics])\n}, train_labels))\n\ntest_dataset = tf.data.Dataset.from_tensor_slices(({\n    'input_ids': np.concatenate([np.array([lyrics['input_ids']]) for lyrics in test_lyrics]),\n    'attention_mask': np.concatenate([np.array([lyrics['attention_mask']]) for ly

In [10]:
# Extract input_ids, attention_mask, and labels from tokenized lyrics
input_ids = np.array([lyrics['input_ids'] for lyrics in subset_data['text']])
attention_mask = np.array([lyrics['attention_mask'] for lyrics in subset_data['text']])
labels = np.array(subset_data['Emotion'])

In [13]:
labels

array(['admiration', 'caring', 'gratitude', ..., 'neutral', 'neutral',
       'neutral'], dtype=object)

In [14]:
# Split the data into train and test sets
input_ids_train, input_ids_test, attention_mask_train, attention_mask_test, y_train, y_test = train_test_split(
    input_ids,
    attention_mask,
    subset_data['Emotion'],
    test_size=0.2,
    random_state=42
)

In [17]:
# Reshape input_ids_train to remove the extra dimension
input_ids_train = input_ids_train.reshape(input_ids_train.shape[0], -1)

# Reshape attention_mask_train to remove the extra dimension
attention_mask_train = attention_mask_train.reshape(attention_mask_train.shape[0], -1)

In [20]:
# Reshape input_ids_train to remove the extra dimension
input_ids_test = input_ids_test.reshape(input_ids_test.shape[0], -1)

# Reshape attention_mask_train to remove the extra dimension
attention_mask_test = attention_mask_test.reshape(attention_mask_test.shape[0], -1)

In [28]:
# Map the labels to integers
y_train_mapped = np.array([emotion_mapping[label] for label in y_train])
y_test_mapped = np.array([emotion_mapping[label] for label in y_test])

In [ ]:
# Shuffle and batch the training dataset
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids_train, 'attention_mask': attention_mask_train}, y_train_mapped)).shuffle(len(y_train)).batch(batch_size)

# Shuffle and batch the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids_test, 'attention_mask': attention_mask_test}, y_test_mapped)).batch(batch_size)

In [29]:
y_test_mapped

array([14,  3, 14,  6, 20,  3,  8,  6,  3,  6, 10,  6,  4,  3,  3,  3,  3,
       10,  3, 11,  3,  3,  3,  3, 11,  3,  7,  6,  6,  3,  4,  8,  3, 10,
        6,  3,  3,  5,  3,  6,  3,  8,  3, 15,  3,  6,  4,  3,  7,  3,  3,
        0,  3,  3,  3,  3,  9,  3,  4,  4,  3,  4,  4,  3,  3,  3,  0, 11,
        3,  8,  6,  3,  7,  3,  4,  6,  3,  3,  6,  3,  4,  8, 20,  3,  3,
        9, 12,  4,  3,  3,  3,  0,  1,  6,  3,  9,  3,  1,  4,  3,  8, 24,
       10,  3,  4,  8,  3,  6, 11,  3,  0,  7,  9,  3,  4,  5,  6,  6,  3,
        3,  6,  0,  6,  3,  3,  3,  3,  3,  3,  6,  3,  6,  6,  8,  4,  0,
       15,  8, 23, 17,  6,  6,  3,  7,  3,  4,  3,  3, 16,  3,  3,  7,  3,
        6,  6,  3,  3,  7,  3,  3,  2,  3,  2,  9,  3,  4,  1,  3, 10,  3,
        4,  3,  3,  1,  6,  3,  6,  6,  6, 10,  3,  6,  3,  3,  3,  2, 13,
        3,  6,  6,  6,  7,  4,  6,  7,  1,  6,  3,  3,  3,  3,  3,  3,  4,
        3,  3,  3,  6,  3,  4, 11,  3,  3,  3,  3,  3,  3,  6,  3,  6,  3,
        4,  3,  3,  3,  4

In [31]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the neural network model
max_length = 512
input_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')
bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # Use the pooled output
output = tf.keras.layers.Dense(8, activation='softmax')(bert_output)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [32]:
# Train the model
epochs = 3
model.fit(train_dataset, epochs=epochs)

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node 'model_3/tf_bert_model_3/bert/encoder/layer_._9/attention/self/MatMul' defined at (most recent call last):
    File "c:\Users\manya\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\manya\Anaconda3\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\manya\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\manya\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
      app.start()
    File "c:\Users\manya\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
      self.io_loop.start()
    File "c:\Users\manya\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\manya\Anaconda3\lib\asyncio\base_events.py", line 534, in run_foreve

In [37]:
"""
# Shuffle and batch the dataset
batch_size = 32
train_dataset = X_train.shuffle(len(subset_data)).batch(batch_size)

# Define the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the neural network model
input_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')
bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # Use the pooled output
output = tf.keras.layers.Dense(8, activation='softmax')(bert_output)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 3
model.fit(train_dataset, epochs=epochs)
"""

"\n# Shuffle and batch the dataset\nbatch_size = 32\ntrain_dataset = X_train.shuffle(len(subset_data)).batch(batch_size)\n\n# Define the BERT model\nbert_model = TFBertModel.from_pretrained('bert-base-uncased')\n\n# Define the neural network model\ninput_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')\nattention_mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')\nbert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # Use the pooled output\noutput = tf.keras.layers.Dense(8, activation='softmax')(bert_output)\n\nmodel = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)\n\n# Compile the model\nmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])\n\n# Train the model\nepochs = 3\nmodel.fit(train_dataset, epochs=epochs)\n"

In [ ]:
# Evaluate the model
model.evaluate(test_dataset)